In [2]:
import dropbox
import glob
import pandas as pd
from bs4 import BeautifulSoup
import os

In [3]:
try:
    df = pd.read_csv("EC1272Z1.dat",sep = '|')
except FileNotFoundError:
    print("file not found")

In [4]:
df.head()

,GEOTYPE,ST,COUNTY,ZIPCODE,GEO_ID,GEO_TTL,SECTOR,NAICS2012,NAICS2012_TTL,RCPSZFE,RCPSZFE_TTL,FOOTID_NAICS,FOOTID_GEO,YEAR,ESTAB,ESTAB_F
0,14,1,0,35004,8610000US35004,"35004 (MOODY, AL)",72,72,Accommodation and food services,1,All establishments,NaN,NaN,2012,15,NaN
1,14,1,0,35004,8610000US35004,"35004 (MOODY, AL)",72,72,Accommodation and food services,2,Establishments operated for the entire year,NaN,NaN,2012,14,NaN
2,14,1,0,35004,8610000US35004,"35004 (MOODY, AL)",72,72,Accommodation and food services,123,Establishments operated entire year with sales...,NaN,NaN,2012,2,NaN
3,14,1,0,35004,8610000US35004,"35004 (MOODY, AL)",72,72,Accommodation and food services,125,Establishments operated entire year with sales...,NaN,NaN,2012,5,NaN
4,14,1,0,35004,8610000US35004,"35004 (MOODY, AL)",72,72,Accommodation and food services,131,Establishments operated entire year with sales...,NaN,NaN,2012,5,NaN


## Percentage of null Values for each column 

In [5]:
df.isnull().sum() / len(df) * 100

GEOTYPE            0.0
ST                 0.0
COUNTY             0.0
ZIPCODE            0.0
GEO_ID             0.0
GEO_TTL            0.0
SECTOR             0.0
NAICS2012          0.0
NAICS2012_TTL      0.0
RCPSZFE            0.0
RCPSZFE_TTL        0.0
FOOTID_NAICS     100.0
FOOTID_GEO       100.0
YEAR               0.0
ESTAB              0.0
ESTAB_F          100.0
dtype: float64

##  Removing Null Columns 

In [7]:
new_df = df[['GEOTYPE', 'ST', 'COUNTY', 'ZIPCODE', 'GEO_ID', 'GEO_TTL', 'SECTOR',
       'NAICS2012', 'NAICS2012_TTL', 'RCPSZFE', 'RCPSZFE_TTL', 'YEAR', 'ESTAB']]

In [8]:
new_df.groupby(["GEO_TTL"])["ST"].count().nlargest(10)

GEO_TTL
14850 (ITHACA, NY)           192
10001 (NEW YORK, NY)         183
95112 (SAN JOSE, CA)         180
65616 (BRANSON, MO)          179
92101 (SAN DIEGO, CA)        179
78666 (SAN MARCOS, TX)       175
94103 (SAN FRANCISCO, CA)    174
10002 (NEW YORK, NY)         171
81301 (DURANGO, CO)          171
93101 (SANTA BARBARA, CA)    170
Name: ST, dtype: int64

In [9]:
#add new column states
states = []
for i in range(len(new_df)):
    states.append(new_df.iloc[i][5].split(",")[1].strip().split(")")[0])

In [10]:
# Add new Column City
cities = []
for i in range(len(new_df)):
    cities.append(new_df["GEO_TTL"].iloc[i].split(",")[0].split("(")[1])

In [12]:
new_df["states"] = states

In [14]:
new_df["city"] = cities

In [15]:
#new_df["NAICS2012_TTL"].unique()

In [16]:
# number of food and Accommodations facilities per city per category 
new_df.groupby(['city','NAICS2012_TTL'])["GEOTYPE"].count().nlargest(10).reset_index(name="count")

,city,NAICS2012_TTL,count
0,HOUSTON,Restaurants and other eating places,1544
1,HOUSTON,Drinking places (alcoholic beverages),1026
2,LOS ANGELES,Restaurants and other eating places,1006
3,CHICAGO,Restaurants and other eating places,916
4,NEW YORK,Restaurants and other eating places,910
5,MIAMI,Restaurants and other eating places,904
6,SAN ANTONIO,Restaurants and other eating places,888
7,CHICAGO,Drinking places (alcoholic beverages),882
8,DALLAS,Restaurants and other eating places,860
9,HOUSTON,Accommodation and food services,811


In [17]:
columns = list(new_df.columns)

In [18]:
new_df.columns= new_df.columns.str.lower()

In [20]:
new_df.rename({"rcpszfe": "revenue", "estab": "num_of_establishments", "geo_ttl": "geo_area_name"},axis=1, inplace=True)

In [21]:
new_df.head()

,geotype,st,county,zipcode,geo_id,geo_area_name,sector,naics2012,naics2012_ttl,revenue,rcpszfe_ttl,year,num_of_establishments,states,city
0,14,1,0,35004,8610000US35004,"35004 (MOODY, AL)",72,72,Accommodation and food services,1,All establishments,2012,15,AL,MOODY
1,14,1,0,35004,8610000US35004,"35004 (MOODY, AL)",72,72,Accommodation and food services,2,Establishments operated for the entire year,2012,14,AL,MOODY
2,14,1,0,35004,8610000US35004,"35004 (MOODY, AL)",72,72,Accommodation and food services,123,Establishments operated entire year with sales...,2012,2,AL,MOODY
3,14,1,0,35004,8610000US35004,"35004 (MOODY, AL)",72,72,Accommodation and food services,125,Establishments operated entire year with sales...,2012,5,AL,MOODY
4,14,1,0,35004,8610000US35004,"35004 (MOODY, AL)",72,72,Accommodation and food services,131,Establishments operated entire year with sales...,2012,5,AL,MOODY


In [85]:
new_df.to_csv("accommodation_food_service" + ".csv")